In [6]:
import cv2
import numpy as np

#El flujo optico esta relacionado con la funcion Lukas-Kanade. Movimiento de puentos especifios o regiones. Tiene que ver una verificacion entre un frame y otro.
#El flujo optico es el seguimiento que se esta realizando
    #1ro detectar esquinas
    #2do se hace el flujo
#Deteccion de esquinas - Shitomasi.

#Control y deteccion de esquina
corner_track_params=dict(maxCorners=10,qualityLevel=0.3,minDistance=7,blockSize=7)

In [7]:
lk_params=dict(winSize=(200,200),maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT,10,0.03))

In [8]:
cap=cv2.VideoCapture(1)
ret,prev_frame=cap.read() #Lectura del frame anterior
prev_frame_gray=cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prevPts=cv2.goodFeaturesToTrack(prev_frame_gray, mask=None, **corner_track_params) #Hacer un rastreo de las mejores caracteristicas
mask=np.zeros_like(prev_frame) #Matriz verificadora de puntos

while True:
    ret, frame=cap.read() #Lectura del frame actual
    frame_gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    nextPts, status,error=cv2.calcOpticalFlowPyrLK(prev_frame_gray, frame_gray,prevPts,None,**lk_params) #El flujo optico se trabaja con la funcion piramidal
    good_new=nextPts[status==1]#Obtener los mejores arrays a nivel de estatus. Frame actual bueno
    good_prev=prevPts[status==1]
    
    for i,(new,prev) in enumerate(zip(good_new, good_prev)):
        x_new,y_new=new.ravel()#ravel aplana la informacion. Matriz a Vector
        x_prev,y_prev=prev.ravel()
        mask=cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3)
        frame=cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)
    img=cv2.add(frame,mask)
    cv2.imshow("Frame",img)
    k=cv2.waitKey(30) & 0xFF
    if k==27:
        break
    pre_gray=frame_gray.copy()
    prevPts=good_new.reshape(-1,1,2)
    
cv2.destroyAllWindows()
cap.realese()

<ipython-input-8-d9dd00c9a24e>:17: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  mask=cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3)
<ipython-input-8-d9dd00c9a24e>:18: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  frame=cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)


AttributeError: 'cv2.VideoCapture' object has no attribute 'realese'